# Pre-training and fine-tuning an LLM on CPU on BEIR datasets with ThirdAI's UDT

In this notebook, we will pre-train a large language model from scratch on the popular BEIR datasets (https://github.com/beir-cellar/beir) using ThirdAIs Universal Deep Transformer (UDT). We will use just the 'Scifact' dataset to demonstrate how UDT can just fine-tune on a small dataset and outperform T5-large trained on a huge corpus. 

This demo shows that one-model for all is not sub-optimal and pre-training on specific downstream datasets is required to get the best results.

While most LLMs cannot be fine-tuned even on a powerful GPU, ThirdAI's UDT can train a billion parameter model on just a moderate CPU in few minutes.

### Import thirdai and activate license

In [ ]:
from thirdai import bolt,licensing

licensing.activate('8E46E4-653FD6-AA2B02-65E265-A4FACA-V3')

### Download and process the dataset into a csv file.

In [ ]:
from thirdai.demos import download_beir_dataset

dataset = "scifact"
unsup_file, sup_train_file, sup_test_file, n_target_classes = download_beir_dataset(dataset)

In the above step, *unsup_file* refers to the corpus file with document id, title and text. We can have even more columns with other metadata for each document. Pre-training with UDT supports two types of columns, strong and weak. For the purpose of this demo, we choose 'title' to be the strong column and 'text' to be the weak column. You can play with different settings shown in the pre-training (model.cold_start()) step.

A couple of sample rows of the *unsup_file* are shown below.

PLEASE NOTE: Currently, UDT's cold_start function requires the DOC_ID to be an integer. We will add support for other formats in a future release.

In [8]:
import pandas as pd

pd.options.display.max_colwidth = 700
pd.read_csv(unsup_file, nrows=2)

,DOC_ID,TITLE,TEXT
0,0,microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging.,alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. a line scan diffusion-weighted magnetic resonance imaging (mri) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient to calculate relative anisotropy and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). to assess effects of prematurity on cerebral white matter development early gestation preterm infants (n = 10) were studied a second time at term. in the central white matter the mean apparent diffusion coeffic...
1,1,induction of myelodysplasia by myeloid-derived suppressor cells.,myelodysplastic syndromes (mds) are age-dependent stem cell malignancies that share biological features of activated adaptive immune response and ineffective hematopoiesis. here we report that myeloid-derived suppressor cells (mdsc) which are classically linked to immunosuppression inflammation and cancer were markedly expanded in the bone marrow of mds patients and played a pathogenetic role in the development of ineffective hematopoiesis. these clonally distinct mdsc overproduce hematopoietic suppressive cytokines and function as potent apoptotic effectors targeting autologous hematopoietic progenitors. using multiple transfected cell models we found that mdsc expansion is driven ...


### Define a UDT model

The column name 'QUERY' has to match with the on in the header in *unsup_test_file*.
The column name 'DOC_ID' should match with the one in the header of the corpus file (*unsup_file*).

In [4]:
model = bolt.UniversalDeepTransformer(
    data_types={
        "QUERY": bolt.types.text(),
        "DOC_ID": bolt.types.categorical(delimiter=':'),
    },
    target="DOC_ID",
    n_target_classes=n_target_classes,
    integer_target=True,
    model_config='./configs/embeddings_and_cold_start.config',
)

### Pre-train (Cold Start) on the *unsup_file*

In the following step, we do the pre-training by specifying the strong and weak columns. For this demo, we use 'TITLE' as the strong column and 'TEXT' as the weak column. We can have more columns in either of the lists. The training time and the test accuracies are shown below. We can see that by just pre-traiing on the Scifact dataset, we get 40% precision@1 which beats T5-large's performance on the same dataset.

In [5]:
model.cold_start(
    filename=unsup_file,
    strong_column_names=["TITLE"],
    weak_column_names=["TEXT"],
    learning_rate=0.001,
    epochs=5,
)

activations = model.evaluate(sup_test_file, metrics=['categorical_accuracy','recall@100'])

loaded data | source 'scifact/unsupervised.csv' | vectors 110213 | batches 54 | time 0s | complete

train | epoch 0 | train_steps 54 | {} | train_batches 54 | time 19s | complete  

train | epoch 1 | train_steps 108 | {} | train_batches 54 | time 18s | complete 

train | epoch 2 | train_steps 162 | {} | train_batches 54 | time 18s | complete 

train | epoch 3 | train_steps 216 | {} | train_batches 54 | time 17s | complete 

train | epoch 4 | train_steps 270 | {} | train_batches 54 | time 17s | complete 

loaded data | source 'scifact/tst_supervised.csv' | vectors 300 | batches 1 | time 0s | complete

evaluate | epoch 5 | train_steps 270 | {categorical_accuracy: 0.4, Recall@100: 0.823} | eval_batches 1 | time 24ms



### Fine-tune on supervised data (OPTIONAL)

If you have supervised data that maps queries to documents, you can further improve the model performance by fine-tuning your pre-trained model on the supervised data.

Please note that in your *sup_train_file* and *sup_test_file* should have the same column names 'QUERY' and 'DOC_ID'.

The training time to fine-tune and the final accuracy are shown below. 

In [6]:
model.train(
    filename=sup_train_file,
    learning_rate=0.001,
    epochs=5,
)

activations = model.evaluate(sup_test_file, metrics=['categorical_accuracy','recall@100'])

loaded data | source 'scifact/trn_supervised.csv' | vectors 809 | batches 1 | time 0s | complete

train | epoch 5 | train_steps 271 | {} | train_batches 1 | time 1s | complete   

train | epoch 6 | train_steps 272 | {} | train_batches 1 | time 1s | complete   

train | epoch 7 | train_steps 273 | {} | train_batches 1 | time 1s | complete   

train | epoch 8 | train_steps 274 | {} | train_batches 1 | time 1s | complete   

train | epoch 9 | train_steps 275 | {} | train_batches 1 | time 1s | complete   

loaded data | source 'scifact/tst_supervised.csv' | vectors 300 | batches 1 | time 0s | complete

evaluate | epoch 10 | train_steps 275 | {categorical_accuracy: 0.583333, Recall@100: 0.891} | eval_batches 1 | time 30ms



### Save and load the model

In [7]:
model.save('./scidocs.model')

model = bolt.UniversalDeepTransformer.load('./scidocs.model')

### Comparisons against T5

| Model | Precision@1 | Recall@100 |
| --- | --- | --- |
| UDT (pre-training + fine-tuning) | 58% | 90% |
|  UDT (just pre-training) |     40%     | 82.3%      |
| T5-large | 39.3%    | 82%        |
| T5-base |  34.7%    | 80%        |